In [2]:
import requests
import json
import gen_ai_web_server.llm_client as llm_client
from bs4 import BeautifulSoup



def build_ner(text, labels):
    return {"prompt":[{"text":text, "labels":labels}]}

def build_ner_eval(text, labels):
    return f"""Generate list of entity labels within <l> </l> tag for the following text: {text}. Current list of labels {labels}. Suggest improvements to this list."""


LLM_TARGET_URL = "http://localhost:5000"
LLM_REQUEST_URL = f"{LLM_TARGET_URL}/request"

NER_TARGET_URL = "http://localhost:5001"
NER_REQUEST_URL = f"{NER_TARGET_URL}/request"

headers = {
    "Content-Type": "application/json",
    "Connection": "keep-alive",

}



QUESTION = "What is nuclear fusion?"
llm = llm_client.Client()

response = llm.send_request([{"role":"user", "content":QUESTION}])
text=(response.get("response")[1]["content"])
print(text)




 Nuclear fusion is a process where two light atomic nuclei combine to form a heavier nucleus, releasing a significant amount of energy in the process. This is the same process that powers the sun and other stars, where hydrogen nuclei fuse to form helium under extreme temperatures and pressures.


The energy released during fusion comes from the fact that the mass of the resulting nucleus is slightly less than the sum of the masses of the original nuclei. This difference in mass is converted into energy according to Einstein's famous equation, E=mc^2, where E is energy, m is mass, and c is the speed of light.


Fusion requires very high temperatures (millions of degrees) to overcome the electrostatic repulsion between the positively charged nuclei. In stars, gravity provides the necessary pressure. On Earth, achieving and maintaining the conditions for fusion is a significant scientific and engineering challenge, and it is the focus of research for potential clean and abundant energy s

In [3]:
final_labels = []
trend = []

THRESHOLD = 0.8
MAX_LOOPS = 3

for i in range(MAX_LOOPS):
    
    lbl_resp = llm.send_request([{"role":"user", "content":build_ner_eval(text, final_labels)}])
    lbl_resp = lbl_resp.get("response")[1]["content"]
    #print(lbl_resp)
    soup = BeautifulSoup(lbl_resp,'lxml')
    l = soup.find_all("l")
    labels = [i.text for i in l]
    
    ner_data = build_ner(text, labels)
    
    response = requests.post(NER_REQUEST_URL, data=json.dumps(ner_data), headers=headers).json()
    entity_list = response.get("response")["entities"]
    filtered_labels = filter(lambda x: x["score"]>THRESHOLD, entity_list)
    final_labels.extend([i["label"] for i in filtered_labels])
    print(len(final_label))
    final_label = list(set(final_labels))   

    
    trend.append(final_label)

for i in trend:
    print(i)
    print(len(i))
    print("--------")



#response = requests.post(NER_REQUEST_URL, data=json.dumps(ner_data), headers=headers).json()

#print(response.get("response"))

ConnectionError: HTTPConnectionPool(host='localhost', port=5001): Max retries exceeded with url: /request (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000205AF06A650>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))